# GANomaly Notebook Results

## Initial Configurations

### Libraries import

In [79]:
import os
import sys
import random
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import umap
import cv2
from IPython.display import display, clear_output
from ipywidgets import interact, IntSlider

sys.path.append("../../")

In [2]:
from utils.metrics import shapiroWilks_test, dagostinoPearson_test, bartlett_test, levene_test, fOneWay_test
from utils.savers import generate_qq_plot

### Experiment selection

In [84]:
experiment_id = "0008"
root_path = "/home/jefelitman/Saved_Models/Anomaly_parkinson/"
for i in sorted(os.listdir("/home/jefelitman/Saved_Models/Anomaly_parkinson/")):
    if experiment_id in i:
        experiment_folder = os.path.join(root_path, i)
experiment_folder

'/home/jefelitman/Saved_Models/Anomaly_parkinson/0008_Ganomaly_2D-64x64x64'

## Quantitative metrics

### Metrics loading

In [4]:
train_metrics = pd.read_csv(os.path.join(experiment_folder, "metrics/train.csv"))
data = train_metrics
np.max(data, axis=0), np.min(data, axis=0), data.to_numpy()[-1]

(epoch          29999.000000
 gen_error         15.549088
 disc_error         8.589197
 accuracy           0.930556
 precision          0.000000
 recall                  NaN
 specificity        0.930556
 f1_score           0.000000
 auc                0.000000
 dtype: float64,
 epoch          0.000000
 gen_error      0.880802
 disc_error     0.767406
 accuracy       0.458333
 precision      0.000000
 recall              NaN
 specificity    0.458333
 f1_score       0.000000
 auc            0.000000
 dtype: float64,
 array([2.99990000e+04, 9.34789002e-01, 7.71247387e+00, 7.08333313e-01,
        0.00000000e+00,            nan, 7.08333313e-01, 0.00000000e+00,
        0.00000000e+00]))

In [5]:
test_metrics = pd.read_csv(os.path.join(experiment_folder, "metrics/test.csv"))
data = test_metrics
np.max(data, axis=0), np.min(data, axis=0), data.to_numpy()[-1]

(epoch          29999.000000
 accuracy           0.894231
 precision          1.000000
 recall             0.875000
 specificity        1.000000
 f1_score           0.933333
 auc                1.000000
 dtype: float64,
 epoch          0.000000
 accuracy       0.105769
 precision      0.272727
 recall         0.034091
 specificity    0.312500
 f1_score       0.060606
 auc            0.107244
 dtype: float64,
 array([2.99990000e+04, 5.76923072e-01, 1.00000000e+00, 5.00000000e-01,
        1.00000000e+00, 6.66666667e-01, 1.00000000e+00]))

### Train Graphics

In [10]:
path = os.path.join(experiment_folder, "outputs/graphics/quantitative/")
portions = 500
rows = 12
columns = 5
for metric in ["gen_error", "disc_error", "accuracy", "specificity"]:
    fig, axs = plt.subplots(rows, columns, figsize=(30, 25))
    fig.suptitle('{} over Epochs'.format(metric))
    for i in range(rows):
        for j in range(columns):
            axs[i, j].plot(train_metrics[metric][(i*columns+j)*portions:(i*columns+j+1)*portions])
            filename = 'train_{}.png'.format(metric)
    fig.savefig(path+filename)
    plt.close(fig)

### Test graphics

In [11]:
path = os.path.join(experiment_folder, "outputs/graphics/quantitative/")
portions = 500
rows = 12
columns = 5
for metric in test_metrics.columns[1:]:
    fig, axs = plt.subplots(rows, columns, figsize=(30, 25))
    fig.suptitle('{} over Epochs'.format(metric))
    for i in range(rows):
        for j in range(columns):
            axs[i, j].plot(test_metrics[metric][(i*columns+j)*portions:(i*columns+j+1)*portions])
            filename = 'test_{}.png'.format(metric)
    fig.savefig(path+filename)
    plt.close(fig)

## Qualitative metrics

### Errors loading

In [12]:
base_path = os.path.join(experiment_folder, "outputs/errors/")
for t in ["encoder", "contextual", "adversarial"]:
    for m in ["train", "test"]:
        classes = ["normal"] if m == "train" else ["normal", "abnormal"]
        for c in classes:
            all_data = "all_{}_{}".format(t, c)
            if all_data not in globals().keys():
                globals()[all_data] = np.r_[[]]
            
            errors = np.load(os.path.join(base_path, t, m, c+".npy"))
            globals()["{}_{}_{}".format(m, t, c)] = errors
            globals()[all_data] = np.concatenate([globals()[all_data], errors])

### Train means and stds

In [13]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal"]:
        data = globals()["train_{}_{}".format(t,c)]
        print("{i} error ({j}): {m} +- {s}".format(
            i = t,
            j = c,
            m = np.mean(data),
            s = np.std(data)
        ))
    print("")

encoder error (normal): 0.0004650423236146 +- 0.0002424867602084916

contextual error (normal): 0.01868695721754597 +- 0.0025115268487707937

adversarial error (normal): 0.0001471521255780317 +- 0.00013049881746628146



### Train statistical tests

In [14]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal"]:
        print("-------------------------------------------------")
        data = globals()["train_{}_{}".format(t,c)]
        norm_dist = stats.norm.rvs(loc=0, scale=1, size=data.shape)
        print("D'agostino-Pearson test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = dagostinoPearson_test(data)
        ))
        print("Shapiro-Wilks test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = shapiroWilks_test(data)
        ))
        print("Barlett test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = bartlett_test(data, norm_dist)
        ))
        print("Levene test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = levene_test(data, norm_dist)
        ))
        print("F1 test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = fOneWay_test(data, norm_dist)
        ))
        print("-------------------------------------------------")
    print("")

-------------------------------------------------
D'agostino-Pearson test for encoder (normal): 2.607277904082647e-05
Shapiro-Wilks test for encoder (normal): 1.787776818673592e-05
Barlett test for encoder (normal): 6.501249481049375e-236
Levene test for encoder (normal): 6.888265445682076e-23
F1 test for encoder (normal): 0.6456902219431118
-------------------------------------------------

-------------------------------------------------
D'agostino-Pearson test for contextual (normal): 0.008343652697901048
Shapiro-Wilks test for contextual (normal): 0.0005736772436648607
Barlett test for contextual (normal): 2.501582181167886e-162
Levene test for contextual (normal): 5.719037837016667e-18
F1 test for contextual (normal): 0.9703867430576745
-------------------------------------------------

-------------------------------------------------
D'agostino-Pearson test for adversarial (normal): 2.8504131787951496e-06
Shapiro-Wilks test for adversarial (normal): 1.3064708070942288e-07
Barle

### Test means and stds

In [15]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        data = globals()["test_{}_{}".format(t,c)]
        print("{i} error ({j}): {m} +- {s}".format(
            i = t,
            j = c,
            m = np.mean(data),
            s = np.std(data)
        ))
    print("")

encoder error (normal): 0.0011111292660643812 +- 0.0006750429368810482
encoder error (abnormal): 0.3347943496297706 +- 0.1325468603884928

contextual error (normal): 0.018606983474455774 +- 0.002537425974949407
contextual error (abnormal): 0.17143731310286306 +- 0.030167396700637

adversarial error (normal): 0.00019274000896984944 +- 0.00016649636796261475
adversarial error (abnormal): 0.26945516132664954 +- 0.43521035222182475



### Test statistical tests

In [16]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        print("-------------------------------------------------")
        data = globals()["test_{}_{}".format(t,c)]
        norm_dist = stats.norm.rvs(loc=0, scale=1, size=data.shape)
        print("D'agostino-Pearson test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = dagostinoPearson_test(data)
        ))
        print("Shapiro-Wilks test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = shapiroWilks_test(data)
        ))
        print("Barlett test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = bartlett_test(data, norm_dist)
        ))
        print("Levene test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = levene_test(data, norm_dist)
        ))
        print("F1 test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = fOneWay_test(data, norm_dist)
        ))
        print("-------------------------------------------------")
    print("")

-------------------------------------------------
D'agostino-Pearson test for encoder (normal): 0.00034359736679840936
Shapiro-Wilks test for encoder (normal): 0.0023234477266669273
Barlett test for encoder (normal): 1.7725997920485505e-42
Levene test for encoder (normal): 5.132401720686475e-06
F1 test for encoder (normal): 0.032980277143216996
-------------------------------------------------
-------------------------------------------------
D'agostino-Pearson test for encoder (abnormal): 2.6092035348239715e-06
Shapiro-Wilks test for encoder (abnormal): 2.7412435883888975e-05
Barlett test for encoder (abnormal): 2.2803283775331808e-54
Levene test for encoder (abnormal): 4.878153894283445e-26
F1 test for encoder (abnormal): 0.00012083541705858026
-------------------------------------------------

-------------------------------------------------
D'agostino-Pearson test for contextual (normal): 0.021697150999042018
Shapiro-Wilks test for contextual (normal): 0.02994360774755478
Barlett 

/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))


### All data means and stds

In [17]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        data = globals()["all_{}_{}".format(t,c)]
        print("{i} error ({j}): {m} +- {s}".format(
            i = t,
            j = c,
            m = np.mean(data),
            s = np.std(data)
        ))
    print("")

encoder error (normal): 0.0005825126767872875 +- 0.00043938272247885283
encoder error (abnormal): 0.3347943496297706 +- 0.1325468603884928

contextual error (normal): 0.018672416536984118 +- 0.0025164446604364326
contextual error (abnormal): 0.17143731310286306 +- 0.030167396700637

adversarial error (normal): 0.0001554408316492713 +- 0.0001388630242109003
adversarial error (abnormal): 0.26945516132664954 +- 0.43521035222182475



### All data statistical tests

In [18]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        print("-------------------------------------------------")
        data = globals()["all_{}_{}".format(t,c)]
        norm_dist = stats.norm.rvs(loc=0, scale=1, size=data.shape)
        print("D'agostino-Pearson test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = dagostinoPearson_test(data)
        ))
        print("Shapiro-Wilks test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = shapiroWilks_test(data)
        ))
        print("Barlett test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = bartlett_test(data, norm_dist)
        ))
        print("Levene test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = levene_test(data, norm_dist)
        ))
        print("F1 test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = fOneWay_test(data, norm_dist)
        ))
        print("-------------------------------------------------")
    print("")

-------------------------------------------------
D'agostino-Pearson test for encoder (normal): 8.380834796717203e-10
Shapiro-Wilks test for encoder (normal): 7.77165762300136e-11
Barlett test for encoder (normal): 7.576582223687879e-265
Levene test for encoder (normal): 1.0620700374518535e-22
F1 test for encoder (normal): 0.6205124784409052
-------------------------------------------------
-------------------------------------------------
D'agostino-Pearson test for encoder (abnormal): 2.6092035348239715e-06
Shapiro-Wilks test for encoder (abnormal): 2.7412435883888975e-05
Barlett test for encoder (abnormal): 4.245291080617993e-51
Levene test for encoder (abnormal): 3.3235040636343257e-15
F1 test for encoder (abnormal): 0.0007164620968219639
-------------------------------------------------

-------------------------------------------------
D'agostino-Pearson test for contextual (normal): 0.015438586712305535
Shapiro-Wilks test for contextual (normal): 0.00030011526541784406
Barlett t

### Train graphics

#### QQ plots

In [19]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/")
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal"]:
        generate_qq_plot(globals()["train_{}_{}".format(t,c)], save_path, "train_{}_{}".format(t,c))

### Test graphics

#### Box plots

In [20]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/")
rows = 1
columns = 3
fig, axs = plt.subplots(rows, columns, figsize=(15, 10))
for i, t in enumerate(["encoder", "contextual", "adversarial"]):
    data = "test_{}_".format(t)
    axs[i].boxplot([globals()[data+"normal"], globals()[data+"abnormal"]], labels=['Normal', 'Abnormal'])
    axs[i].set_title("{} errors".format(t))

filename = 'test_boxplot.png'
fig.savefig(save_path+filename)
plt.close(fig)

#### QQ plots

In [21]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/")
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        generate_qq_plot(globals()["test_{}_{}".format(t,c)], save_path, "test_{}_{}".format(t,c))

### All data graphics

#### Box plots

In [22]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/")
rows = 1
columns = 3
fig, axs = plt.subplots(rows, columns, figsize=(15, 10))
for i, t in enumerate(["encoder", "contextual", "adversarial"]):
    data = "all_{}_".format(t)
    axs[i].boxplot([globals()[data+"normal"], globals()[data+"abnormal"]], labels=['Normal', 'Abnormal'])
    axs[i].set_title("{} errors".format(t))

filename = 'all_data_boxplot.png'
fig.savefig(save_path+filename)
plt.close(fig)

#### QQ plots

In [23]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/")
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        generate_qq_plot(globals()["all_{}_{}".format(t,c)], save_path, "all_data_{}_{}".format(t,c))

## Visual results

### UMAP latent vectors

In [24]:
base_path = os.path.join(experiment_folder, "outputs/latent_vectors/")
for t in ["in", "out"]:
    for m in ["train", "test"]:
        classes = ["normal"] if m == "train" else ["normal", "abnormal"]
        for c in classes:
            all_data = "all_{}_{}".format(t, c)
            if all_data not in globals().keys():
                globals()[all_data] = []
            
            data = "{}_{}_{}".format(m, t, c)
            globals()[data] = []
            path = os.path.join(base_path, t, m, c)
            for file in sorted(os.listdir(path)):
                vector = np.load(os.path.join(path, file))
                globals()[data].append(vector)
                globals()[all_data].append(vector)
            globals()[data] = np.r_[globals()[data]]
for t in ["in", "out"]:
    for c in ["normal", "abnormal"]:
        all_data = "all_{}_{}".format(t, c)
        globals()[all_data] = np.r_[globals()[all_data]]

In [26]:
save_path = os.path.join(experiment_folder, "outputs/graphics/visuals/")
for t in ["in", "out"]:
    for m in ["train", "test", "all"]:
        classes = ["normal"] if m == "train" else ["normal", "abnormal"]
        globals()["{}_mapped_{}".format(m, t)] = umap.UMAP(random_state = 8128).fit_transform(np.concatenate(
                [globals()["{}_{}_{}".format(m, t, c)] for c in classes], axis=0
            )
        )
        
        divisor = globals()["{}_{}_normal".format(m, t)].shape[0]
        plt.scatter(globals()["{}_mapped_{}".format(m, t)][:divisor,0],
            globals()["{}_mapped_{}".format(m, t)][:divisor,1], color="blue", s=5, label="Normal"
        )
        if "abnormal" in classes:
            plt.scatter(globals()["{}_mapped_{}".format(m, t)][divisor:,0],
                globals()["{}_mapped_{}".format(m, t)][divisor:,1], color="red", s=5, label="Abnormal"
            )
        plt.legend()
        plt.title("UMAP for {}put data in {}".format(t, m))
        filename = '{}_umap_{}put.png'.format(m, t)
        plt.savefig(save_path+filename)
        plt.close()

### Video comparision

In [83]:
for i in range(0, 88):
    video_index = i
    mode = "test"
    video_class = "abnormal"

    base_path = os.path.join(experiment_folder, "outputs/samples/")
    for t in ["real", "fake", "substraction"]:
        path = os.path.join(base_path, t, mode, video_class)
        video = sorted(os.listdir(path))[video_index]
        data = "video_{}".format(t)
        globals()[data] = []
        for frame in sorted(os.listdir(os.path.join(path, video))):
            globals()[data].append(cv2.cvtColor(
                cv2.imread(
                    os.path.join(path, video, frame)
                ), cv2.COLOR_BGR2RGB
            ))
        globals()[data] = np.r_[globals()[data]]  

    assert video_real.shape[0] == video_fake.shape[0] == video_substraction.shape[0]

    widget = IntSlider(min=1, max=video_real.shape[0], step=1)

    interact(lambda frame: plt.imshow(video_real[frame-1]), frame=widget)
    interact(lambda frame: plt.imshow(video_fake[frame-1]), frame=widget)
    interact(lambda frame: plt.imshow(video_substraction[frame-1]), frame=widget)
    input("indice actual: {}".format(i))
    clear_output()

interactive(children=(IntSlider(value=1, description='frame', max=64, min=1), Output()), _dom_classes=('widget…

interactive(children=(IntSlider(value=1, description='frame', max=64, min=1), Output()), _dom_classes=('widget…

interactive(children=(IntSlider(value=1, description='frame', max=64, min=1), Output()), _dom_classes=('widget…

KeyboardInterrupt: Interrupted by user